In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import time,datetime

import pandas as pd
import random

In [3]:
%load_ext autoreload

%autoreload 3

%reload_ext autoreload

In [5]:
def path_check(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"폴더 생성: {folder_path}")
    else:
        print(f"폴더가 이미 존재함: {folder_path}")
    return folder_path

In [6]:
curr_path = os.getcwd()
util_path = path_check(os.path.join(curr_path,'utils'))
log_path = path_check(os.path.join(curr_path,'logs'))
driver_path = os.path.join(util_path,'chromedriver.exe')
category_raw_path = os.path.join(curr_path,'category.txt')

print(curr_path)
print(driver_path)
print(category_raw_path)

폴더가 이미 존재함: C:\Users\royal\OneDrive\바탕 화면\NLP_workbench\utils
폴더가 이미 존재함: C:\Users\royal\OneDrive\바탕 화면\NLP_workbench\logs
C:\Users\royal\OneDrive\바탕 화면\NLP_workbench
C:\Users\royal\OneDrive\바탕 화면\NLP_workbench\utils\chromedriver.exe
C:\Users\royal\OneDrive\바탕 화면\NLP_workbench\category.txt


In [4]:
def logging(context,log_path = log_path):
    with open (os.path.join(log_path,'log.txt'),'a') as f:
        f.write(f'{datetime.datetime.today().strftime("%Y/%m/%d %H:%M:%S")} : {context}\n')

In [7]:
with open(category_raw_path,'r') as f:
    raw_lst = f.readlines()[0].split(',')
    category_lst = [category.strip().lower() for category in raw_lst]
print(category_lst)

['renaissance']


In [8]:
def write_df(url_lst,category):
    folder_path = os.path.join(curr_path,'urls')
    url_dct = {'url':url_lst}
    path_check(folder_path)
    df = pd.DataFrame(url_dct)
    df.to_csv(os.path.join(folder_path,category+'.csv'))
    logging(f'Write {category}.csv at {folder_path}')

In [9]:
def scraping(category,driver_path):
    collected_url = []
    cnt = 0
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    driver.get('https://artsandculture.google.com/category/art-movement?hl=en')
    SCROLL_PAUSE_SEC = 1.5
    actions = ActionChains(driver)
    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)
        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    time.sleep(5)
    for i in range(1,125):
        element_xpath = f'//*[@id="tab_pop"]/div/ul/li[{i}]/a/span/span/span[1]'
        element = WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.XPATH, element_xpath)))
        if element.text.strip().lower() == category:
            # 상위 a 태그로 이동
            parent_element = driver.find_element(By.XPATH, element_xpath.split('/span')[0])
            # href 속성 가져오기
            category_url = parent_element.get_attribute('href')
            driver.get(category_url)
            break
    else : return False
    time.sleep(random.randint(5, 10))
    element = driver.find_element(By.CLASS_NAME, "ldhC4e.PJLMUc")
    element.click()
    time.sleep(5)
    while True:
    # 현재 페이지의 URL 얻기
        initial_url = driver.current_url
        logging(f'{category} | {initial_url}')
        collected_url.append(initial_url)
        # 다음 버튼 클릭
        cnt+=1
        if(cnt%100==0):
            print(cnt)
        else:
            print(cnt, end = ' ')
        actions.send_keys(Keys.ARROW_RIGHT).perform()
        time.sleep(7)
        changed_url = driver.current_url
        if (cnt>=1300) or (changed_url == initial_url):            
            print('No more pages.')
            driver.quit()
            write_df(collected_url,category)
            return True

In [ ]:
for category in category_lst:
    res = scraping(category,driver_path)
    print(f'{category} finished, result{res}')
print('all_job_finished')